In [ ]:
from airline_revenue_analytics.viz.charts import apply_style, PLOT_COLORS
apply_style()
PASS_COLOR = "#D9F2E6"
FAIL_COLOR = "#FCE4E4"
NEG_BG_COLOR = FAIL_COLOR


In [ ]:
# Project paths (booking pipeline)
from pathlib import Path
import sys

def find_repo_root(start: Path) -> Path:
    for p in [start] + list(start.parents):
        if (p / "pyproject.toml").exists() and (p / "src" / "airline_revenue_analytics").exists():
            return p
    return start

REPO_ROOT = find_repo_root(Path.cwd())
PROJECT_ROOT = REPO_ROOT
SRC_ROOT = REPO_ROOT / "src"
if str(SRC_ROOT) not in sys.path:
    sys.path.append(str(SRC_ROOT))

from airline_revenue_analytics.config import get_paths

PATHS = get_paths("booking")
DATA_DIR = REPO_ROOT / "data"
RAW_DIR = PATHS.data_raw
DB_PATH = PATHS.db_path
OUTPUT_DIR = PATHS.outputs_root
FIG_DIR = PATHS.figures
TAB_DIR = PATHS.tables
ART_DIR = PATHS.artifacts

def _rel(p: Path) -> str:
    try:
        return str(Path(p).resolve().relative_to(REPO_ROOT))
    except Exception:
        return Path(p).name

print("REPO_ROOT:", REPO_ROOT.name)
print("DB_PATH:", _rel(DB_PATH))
print("OUTPUT_DIR:", _rel(OUTPUT_DIR))


# 05 - Modeling Loop B (Steps 5–7, stronger models + iteration)
Goal: train stronger models (regularised linear + boosted tree) on the SAME split,
export metrics/predictions/plots, and compare with Loop A for Step 8.5 evidence.

Outputs:
- `outputs/booking/tables/model_loopB_results.csv`
- `outputs/booking/tables/model_comparison_all.csv`                # Loop A + Loop B
- `outputs/booking/tables/table_7_1_predictions_loopB.csv`
- `outputs/booking/tables/table_5_2_ridge_coefficients_loopB.csv`
- `outputs/booking/figures/figure_7_1_residuals_vs_fitted_loopB.png`
- `outputs/booking/figures/figure_7_1_actual_vs_pred_loopB.png`
- `outputs/booking/artifacts/pipe_ridge_loopB.joblib`
- `outputs/booking/artifacts/pipe_hgbr_loopB.joblib`


In [ ]:
# Imports & paths
import sys, pathlib, numpy as np, pandas as pd
import matplotlib; import matplotlib.pyplot as plt

# ensure "src" importable when running from notebooks/
sys.path.append(str(PROJECT_ROOT))


from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge
# HistGradientBoostingRegressor: enable on very old sklearn
try:
    from sklearn.experimental import enable_hist_gradient_boosting  # noqa: F401
except Exception:
    pass
try:
    from sklearn.ensemble import HistGradientBoostingRegressor
    HGBR_AVAILABLE = True
except Exception:
    from sklearn.ensemble import GradientBoostingRegressor  # fallback
    HGBR_AVAILABLE = False

from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import joblib

OUT_DIR = OUTPUT_DIR
FIG_DIR = OUT_DIR / "figures"
TAB_DIR = OUT_DIR / "tables"
ART_DIR = OUT_DIR / "artifacts"

SEED = 42
np.random.seed(SEED)

print("FIG_DIR:", _rel(FIG_DIR))
print("TAB_DIR:", _rel(TAB_DIR))
print("ART_DIR:", _rel(ART_DIR), "| HGBR_AVAILABLE:", HGBR_AVAILABLE)


In [ ]:
# Load dataset built in 02 and split file produced in 03
df = pd.read_parquet(TAB_DIR / "booking_model_df.parquet")
split_ids = pd.read_csv(TAB_DIR / "table_4_3_1_split_ids.csv")

# Target & features (same as 03/04)
target = "log_total_amount"
num_cols = [
    "n_segments", "sum_sched_duration_min", "avg_sched_duration_min",
    "max_sched_duration_min", "share_premium_cabin", "max_cabin_index",
    "has_longhaul", "n_unique_routes", "avg_booking_lead_days"
]

# ---------- Reconstruct split (use index if present; otherwise map by book_ref) ----------
if "index" in split_ids.columns:
    idx_train = split_ids.loc[split_ids["split"].eq("train"), "index"].to_numpy()
    idx_test  = split_ids.loc[~split_ids["split"].eq("train"), "index"].to_numpy()
else:
    idx_map = df.reset_index()[["index", "book_ref"]]
    merged  = split_ids.merge(idx_map, on="book_ref", how="left", validate="one_to_one")
    idx_train = merged.loc[merged["split"].eq("train"), "index"].to_numpy()
    idx_test  = merged.loc[~merged["split"].eq("train"), "index"].to_numpy()

# Categorical: top-K route mapping (train-only)
cat_raw = df["primary_route_code"].astype("string").fillna("Unknown")
top_k = 20
top_routes = cat_raw.loc[idx_train].value_counts().index[:top_k].tolist()
df["primary_route_code_top"] = np.where(cat_raw.isin(top_routes), cat_raw, "Other")
cat_cols = ["primary_route_code_top"]

# Compose X/y
X_all = df[num_cols + cat_cols].copy()
y_all = df[target].copy()

X_train, y_train = X_all.loc[idx_train], y_all.loc[idx_train]
X_test,  y_test  = X_all.loc[idx_test],  y_all.loc[idx_test]

X_train.shape, X_test.shape

# Convenience aliases (target already log scale)
y_train_log, y_test_log = y_train, y_test


In [ ]:
# Build preprocessor (fit later inside each pipeline)
try:
    ohe = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
except TypeError:
    ohe = OneHotEncoder(handle_unknown="ignore", sparse=False)

pre = ColumnTransformer([
    ("num", Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler())
    ]), num_cols),
    ("cat", Pipeline([
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("ohe", ohe)
    ]), cat_cols),
])

pre


In [ ]:
# Version-robust evaluation (handles old sklearn without squared=)
def evaluate(y_true, y_pred):
    y_true = np.asarray(y_true).ravel()
    y_pred = np.asarray(y_pred).ravel()
    r2 = float(r2_score(y_true, y_pred))
    try:
        rmse = float(mean_squared_error(y_true, y_pred, squared=False))
    except TypeError:
        rmse = float(mean_squared_error(y_true, y_pred) ** 0.5)
    mae = float(mean_absolute_error(y_true, y_pred))
    return {"R2": r2, "RMSE": rmse, "MAE": mae}


In [ ]:
# Ridge (regularised linear) – modest grid
ridge = Ridge(random_state=SEED)
ridge_grid = {"model__alpha": np.logspace(-3, 3, 13)}

pipe_ridge = Pipeline([("pre", pre), ("model", ridge)])
gs_ridge = GridSearchCV(
    estimator=pipe_ridge,
    param_grid=ridge_grid,
    scoring="neg_root_mean_squared_error",  # works if supported; fallback handled below
    cv=3,
    n_jobs=1,
    refit=True
)
try:
    gs_ridge.fit(X_train, y_train)
except ValueError:
    # Older sklearn may not have neg_root_mean_squared_error; fallback to neg_mean_squared_error
    gs_ridge = GridSearchCV(
        estimator=pipe_ridge,
        param_grid=ridge_grid,
        scoring="neg_mean_squared_error",
        cv=3,
        n_jobs=1,
        refit=True
    )
    gs_ridge.fit(X_train, y_train)

best_ridge = gs_ridge.best_estimator_
pred_ridge = best_ridge.predict(X_test)
m_ridge = {"model": "Ridge"} | evaluate(y_test, pred_ridge)
m_ridge["alpha"] = float(gs_ridge.best_params_["model__alpha"])

# Export coefficients (post-OHE feature names from TRAIN)
ohe_train = best_ridge.named_steps["pre"].named_transformers_["cat"].named_steps["ohe"]
feat_names = num_cols + list(ohe_train.get_feature_names_out(cat_cols))
coefs = best_ridge.named_steps["model"].coef_
coef_df = pd.DataFrame({"feature": feat_names, "coef": coefs})
coef_df["abs_coef"] = coef_df["coef"].abs()
coef_df.sort_values("abs_coef", ascending=False).to_csv(TAB_DIR/"table_5_2_ridge_coefficients_loopB.csv", index=False)

# Save artifact
joblib.dump(best_ridge, ART_DIR/"pipe_ridge_loopB.joblib")

m_ridge, coef_df.head(8)


In [ ]:
if HGBR_AVAILABLE:
    gbr = HistGradientBoostingRegressor(
        random_state=SEED,
        max_depth=None,
        max_leaf_nodes=31,
        learning_rate=0.1,
        min_samples_leaf=20
    )
    param_grid = {
        "model__max_leaf_nodes": [31, 63],
        "model__learning_rate": [0.05, 0.1],
        "model__min_samples_leaf": [10, 20, 40],
    }
else:
    # Fallback to classic GradientBoostingRegressor
    from sklearn.ensemble import GradientBoostingRegressor
    gbr = GradientBoostingRegressor(
        random_state=SEED,
        learning_rate=0.1,
        n_estimators=300,
        max_depth=3
    )
    param_grid = {
        "model__learning_rate": [0.05, 0.1],
        "model__n_estimators": [200, 300, 500],
        "model__max_depth": [2, 3],
    }

pipe_gbr = Pipeline([("pre", pre), ("model", gbr)])
gs_gbr = GridSearchCV(
    estimator=pipe_gbr,
    param_grid=param_grid,
    scoring="neg_mean_squared_error",  # use MSE for broad compatibility
    cv=3,
    n_jobs=1,
    refit=True
)
gs_gbr.fit(X_train, y_train)

best_gbr = gs_gbr.best_estimator_
pipe_hgb = best_gbr
pred_gbr = best_gbr.predict(X_test)
m_gbr = {"model": "HistGBR" if HGBR_AVAILABLE else "GBR"} | evaluate(y_test, pred_gbr)
m_gbr["best_params"] = str(gs_gbr.best_params_)

# Save artifact
joblib.dump(best_gbr, ART_DIR/"pipe_hgbr_loopB.joblib")

m_gbr


In [ ]:
# Loop B results table
loopB_results = pd.DataFrame([m_ridge, m_gbr]).sort_values("RMSE")
loopB_results.to_csv(TAB_DIR/"model_loopB_results.csv", index=False)

# Combine with Loop A (if file exists)
try:
    loopA = pd.read_csv(TAB_DIR/"model_baselines_loopA.csv")
    cmp_all = pd.concat([loopA.assign(loop="A"), loopB_results.assign(loop="B")], ignore_index=True)
    cmp_all.sort_values(["RMSE","loop"]).to_csv(TAB_DIR/"model_comparison_all.csv", index=False)
    cmp_all
except FileNotFoundError:
    print("Loop A metrics not found; only Loop B results are saved.")
    loopB_results


In [ ]:
# Pick the best model within Loop B
bestB = m_ridge if m_ridge["RMSE"] <= m_gbr["RMSE"] else m_gbr
best_name = bestB["model"]
best_pipe = best_ridge if best_name == "Ridge" else best_gbr
best_pred = pred_ridge if best_name == "Ridge" else pred_gbr

# Residuals vs Fitted
resid = y_test - best_pred
plt.figure()
plt.scatter(best_pred, resid, s=12, alpha=0.7)
plt.axhline(0, linestyle="--")
plt.title(f"Residuals vs Fitted (Test) – {best_name} (Loop B)")
plt.xlabel("Fitted"); plt.ylabel("Residuals")
plt.tight_layout()
plt.savefig(FIG_DIR/"figure_7_1_residuals_vs_fitted_loopB.png", dpi=150)
plt.show()

# Actual vs Predicted
plt.figure()
plt.scatter(y_test, best_pred, s=12, alpha=0.7)
plt.title(f"Actual vs Predicted (Test) – {best_name} (Loop B)")
plt.xlabel("Actual (log_total_amount)"); plt.ylabel("Predicted")
plt.tight_layout()
plt.savefig(FIG_DIR/"figure_7_1_actual_vs_pred_loopB.png", dpi=150)
plt.show()

# Save per-row predictions (evidence)
pred_df = pd.DataFrame({
    "index": X_test.index,
    "book_ref": df.loc[X_test.index, "book_ref"].values,
    "y_true": y_test.values,
    f"y_pred_{best_name}_loopB": best_pred
})
pred_df.to_csv(TAB_DIR/"table_7_1_predictions_loopB.csv", index=False)

best_name, loopB_results.sort_values("RMSE")


**Summary (Loop B):**
- Trained stronger models (Ridge + HistGBR/GBR) on the SAME split as Loop A.
- Exported metrics & predictions; produced residual plots; dumped fitted pipelines.
- Wrote `model_comparison_all.csv` for Step 8.5 (iteration) comparison.

**Next:** open `06_interpretation_compare.ipynb` for Step 8:
- compare Loop A vs Loop B (tables/plots),
- discuss business interpretation and next actions,
- optionally add error-by-segment/cabin/route buckets for insights.


In [ ]:
# ---- 7.2 / Table XV: RUB errors for HistGBR (champion) ----
import numpy as np, pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error

def rmse_score(y_true, y_pred):
    try:
        return float(mean_squared_error(y_true, y_pred, squared=False))
    except TypeError:
        return float(np.sqrt(mean_squared_error(y_true, y_pred)))

def backtransform_with_lognormal_correction(pipeline, X_tr, y_tr_log, X_te):
    yhat_tr_log = pipeline.predict(X_tr)
    sigma2 = np.mean((y_tr_log - yhat_tr_log)**2)
    yhat_te_log = pipeline.predict(X_te)
    return np.exp(yhat_te_log + 0.5 * sigma2)

y_test_rub = np.exp(y_test_log)
yhat_rub = backtransform_with_lognormal_correction(pipe_hgb, X_train, y_train_log, X_test)
rub_row = pd.DataFrame([[
    "HistGradientBoosting (Loop B)",
    rmse_score(y_test_rub, yhat_rub),
    mean_absolute_error(y_test_rub, yhat_rub),
    float(np.median(np.abs(y_test_rub - yhat_rub)))
]], columns=["Model","RMSE_RUB","MAE_RUB","Median_AE_RUB"])

# Append/merge with Loop A RUB table
path_a = TAB_DIR/"table_7_2_RUB_errors_loopA.csv"
rub_all = pd.concat([pd.read_csv(path_a), rub_row], ignore_index=True) if path_a.exists() else rub_row
rub_all.to_csv(TAB_DIR/"table_7_2_RUB_errors_all.csv", index=False)
rub_all


In [ ]:
# ---- 8.2.1: Test-set permutation importance for HistGBR ----
from sklearn.inspection import permutation_importance

pre = pipe_hgb.named_steps.get("pre", pipe_hgb.named_steps.get("preprocessor"))
feat_names = getattr(pre, "get_feature_names_out", lambda: np.array([]))().tolist()
r = permutation_importance(pipe_hgb, X_test, y_test_log, n_repeats=5, random_state=42, scoring="r2")

# Fallback to generic names if transformer cannot provide names
if len(feat_names) != len(r.importances_mean):
    feat_names = [f"f{i}" for i in range(len(r.importances_mean))]

pi = (pd.DataFrame({"feature": feat_names, "importance_mean": r.importances_mean, "importance_std": r.importances_std})
        .sort_values("importance_mean", ascending=False))
pi.to_csv(TAB_DIR/"table_8_2_1_histgbr_permutation_importance.csv", index=False)
pi.head(20)


In [ ]:
# ---- 8.2.2: PDP/ICE for Lead, Duration, Cabin Index; 2-D PDP for (Lead, Duration) ----
from sklearn.inspection import PartialDependenceDisplay
import matplotlib.pyplot as plt

one_d = [("avg_booking_lead_days",), ("avg_sched_duration_min",), ("max_cabin_index",)]
for f in one_d:
    PartialDependenceDisplay.from_estimator(pipe_hgb, X_test, [f], kind="both")
    plt.title(f"PDP/ICE — {f[0]} (HistGBR, test-set)")
    plt.show()

PartialDependenceDisplay.from_estimator(pipe_hgb, X_test,
    [("avg_booking_lead_days", "avg_sched_duration_min")])
plt.title("2D PDP — (Lead, Duration) (HistGBR, test-set)")
plt.show()
